In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from loguru import logger

In [4]:
import os
import sys
sys.path.append('..')

In [5]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.evaluation import run_resample_experiment, _select_significant_entries, _compute_prec_rec_f1
from pals.common import *
from pals.feature_extraction import DataSource
from pals.PALS import PALS
from pals.ORA import ORA

2020-01-10 12:08:23.159 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2020-01-10 12:08:23.160 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2020-01-10 12:08:23.198 | WARNING  | pals.reactome:get_neo4j_driver:27 - Failed to connect to graph database: Failed to establish connection to ('127.0.0.1', 7687) (reason [Errno 111] Connection refused)
2020-01-10 12:08:23.198 | WARNING  | pals.reactome:<module>:34 - Driver initialisation failed. PALS will run without Reactome support.


# HAT Data Analysis

This notebook is used to generate resampled data and run the different methods for comparison in the manuscript. Results are analysed in `PALS_HAT_experiment_evaluation.ipynb`

### Load data

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [6]:
token = get_pimp_API_token_from_env()

In [7]:
analysis_id_plasma = 636
int_df_plasma, annotation_df_plasma, experimental_design_plasma = download_from_pimp(token, PIMP_HOST, analysis_id_plasma, 'kegg')

2020-01-10 12:08:23.261 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: /tmp/pimp_analysis_636.p


In [8]:
analysis_id_csf = 635
int_df_csf, annotation_df_csf, experimental_design_csf = download_from_pimp(token, PIMP_HOST, analysis_id_csf, 'kegg')

2020-01-10 12:08:23.325 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: /tmp/pimp_analysis_635.p


### Create Data Sources

In [9]:
database_name = DATABASE_PIMP_KEGG
ds_plasma = DataSource(int_df_plasma, annotation_df_plasma, experimental_design_plasma, database_name)
ds_csf = DataSource(int_df_csf, annotation_df_csf, experimental_design_csf, database_name)

2020-01-10 12:08:23.366 | DEBUG    | pals.feature_extraction:__init__:42 - Using PiMP_KEGG as database
2020-01-10 12:08:23.367 | DEBUG    | pals.feature_extraction:get_database:105 - Loading /home/joewandy/git/PALS/pals/data/PiMP_KEGG.json.zip
2020-01-10 12:08:23.390 | DEBUG    | pals.feature_extraction:__init__:55 - Mapping pathway to unique ids
2020-01-10 12:08:23.395 | DEBUG    | pals.feature_extraction:__init__:69 - Creating dataset to pathway mapping
2020-01-10 12:08:23.881 | DEBUG    | pals.feature_extraction:__init__:97 - Computing unique id counts
2020-01-10 12:08:23.888 | DEBUG    | pals.feature_extraction:__init__:42 - Using PiMP_KEGG as database
2020-01-10 12:08:23.889 | DEBUG    | pals.feature_extraction:get_database:105 - Loading /home/joewandy/git/PALS/pals/data/PiMP_KEGG.json.zip
2020-01-10 12:08:23.913 | DEBUG    | pals.feature_extraction:__init__:55 - Mapping pathway to unique ids
2020-01-10 12:08:23.919 | DEBUG    | pals.feature_extraction:__init__:69 - Creating datas

In [10]:
ds_plasma.get_measurements().shape

(15584, 60)

In [11]:
ds_csf.get_measurements().shape

(8154, 57)

# Run PALS Experiments

In [12]:
proportions = np.array([0.25, 0.50, 0.75])
num_iterations = 500
plage_weight = 5
hg_weight = 1
gsea_resamples = 1000

# pbar = True
# parallel = False

pbar = False
parallel = True

In [13]:
results = {}

#### Disable debug logging

In [14]:
set_log_level_info()

#### Run on Plasma samples

In [15]:
data_source = ds_plasma
experiment_name = 'plasma'
case = 'Stage1'
control = 'Control'
significant_column = 'PiMP_KEGG Stage1/Control comb_p'

In [16]:
res = run_resample_experiment(experiment_name, data_source, case, control, proportions, significant_column, num_iterations, plage_weight, hg_weight, gsea_resamples, 
                              pbar=pbar, parallel=parallel)
results[experiment_name] = res

2020-01-10 12:08:24.467 | INFO     | pals.evaluation:run_resample_experiment:207 - prop=0.25 n_sample=3896 PALS experiment=plasma case=Stage1 control=Control
2020-01-10 12:37:37.977 | INFO     | pals.evaluation:run_resample_experiment:207 - prop=0.50 n_sample=7792 PALS experiment=plasma case=Stage1 control=Control
2020-01-10 13:47:01.544 | INFO     | pals.evaluation:run_resample_experiment:207 - prop=0.75 n_sample=11688 PALS experiment=plasma case=Stage1 control=Control


#### Run on CSF samples

In [17]:
data_source = ds_csf
experiment_name = 'csf'
case = 'Stage_2'
control = 'Control'
significant_column = 'PiMP_KEGG Stage_2/Control comb_p'

In [18]:
res = run_resample_experiment(experiment_name, data_source, case, control, proportions, significant_column, num_iterations, plage_weight, hg_weight, gsea_resamples, 
                              pbar=pbar, parallel=parallel)
results[experiment_name] = res

2020-01-10 15:45:08.029 | INFO     | pals.evaluation:run_resample_experiment:207 - prop=0.25 n_sample=2038 PALS experiment=csf case=Stage_2 control=Control
2020-01-10 16:01:25.307 | INFO     | pals.evaluation:run_resample_experiment:207 - prop=0.50 n_sample=4077 PALS experiment=csf case=Stage_2 control=Control
2020-01-10 16:38:46.018 | INFO     | pals.evaluation:run_resample_experiment:207 - prop=0.75 n_sample=6115 PALS experiment=csf case=Stage_2 control=Control


#### Save Results

In [19]:
save_obj(results, os.path.join('test_data', 'HAT', 'HAT_results.p'))